In [1]:
!python --version

Python 3.9.0


In [6]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [11]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    print('Colunms: {}'.format(len(df.columns)))

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    print('SD: {}'.format(np.std(df.duration)))
    
    orig_len=len(df)
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    new_len=len(df)
    change_per=(new_len/orig_len)*100
    print('change%: {}'.format(change_per))

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [12]:
df_train = read_dataframe('dataset/yellow_tripdata_2022-01.parquet')
df_test = read_dataframe('dataset/yellow_tripdata_2022-02.parquet')

Colunms: 19
SD: 46.445295712725304
change%: 98.27547930522405
Colunms: 19
SD: 47.2639328360818
change%: 97.9444397269143


In [19]:
dv = DictVectorizer()
categorical = ['PULocationID', 'DOLocationID']
train_dicts=(df_train[categorical]).to_dict(orient='records')
X_train=dv.fit_transform(train_dicts)

print(X_train.shape)

target='duration'
y_train=df_train[target].values

lr=LinearRegression()
lr.fit(X_train, y_train)

y_pred=lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

(2421440, 515)


6.986191065500608

In [21]:
val_dicts = df_test[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

y_val = df_test[target].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.786408015215065